In [1]:
!pip install --upgrade cvxpy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install botorch

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

from cvxpy import *
import pandas as pd
import seaborn as sns

import os
import torch

In [4]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


tkwargs = {
    "dtype": torch.double,
    #"device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}
SMOKE_TEST = os.environ.get("SMOKE_TEST")

from __future__ import annotations

import math
from abc import ABC, abstractmethod
from math import pi
from typing import Optional

from botorch.test_functions.base import (
    ConstrainedBaseTestProblem,
    MultiObjectiveTestProblem,
)

In [5]:
#dataset macronutrients and micronutrients and others.

#Costs of raws
Costos=np.array([141.24,144.24,147.25,132.22,132.22,195.32,342.57,120.2,30.05,1803.03,111.18,336.56,152.65,150.25,159.27,136.73,300.5])
#CF
CF=np.array([4.5,2.8,2.5,24.7,6.1,5.6,1,8,0,0,22.5,0,17.8,14.5,5.7,2.3,0])
#Ca
Ca=np.array([0.06,0.04, 0.02, 1.75, 0.24,0.29,4.50,0.16,38.3,0.04, 0.35, 0, 0.98, 0.23, 0.10, 0.05, 32])
#AP
AP=np.array([0.13, 0.18, 0.05, 0.26, 0.03, 0.19, 2.45, 0.22, 0,0,0.17, 0, 0.04, 0.13, 0.15, 0.15,0])
#DM
DM=np.array([90.2,89.4, 86.3,91.2, 88.8, 88, 92, 88.6, 98, 98.5, 89.3, 0, 89.7, 90.8, 86.7, 89.4, 99.4])
#CP
CP=np.array([11.3, 11.6, 7.7, 16.7, 2.5, 44, 62.4, 19, 0, 95, 30.5, 0, 10.1,30.7, 21.5, 8.9, 0])
#Energy
Energy=np.array([14.7446, 15.4048, 14.9983,5.7664, 10.9456,15.3092,14.4188,12.5758,0,16.4095,9.9224,17.37,7.9859,  14.363 ,  14.4828,  15.2728, 0])
#L
L=np.array([0.4,0.33,0.22, 0.73, 0.09, 2.88, 4.75, 0.62, 0, 78, 1.06, 0, 0.59, 1.46, 1.55, 0.34, 0])
#MC
MC=np.array([0.43,0.46,0.33,0.45,0.06,1.28,2.36,0.83,0,0,1.25,0,0.22,0.66,0.56,0.37,0])
#T
T=np.array([0.37,0.34,0.27,0.7,0.07,1.75,2.65,0.74,0,0,1.06,0,0.47,0.99,0.82,0.3,0])
#Tp
Tp=np.array([0.13,0.13,0.06,0.31,0.02,0.59,0.65,0.13,0,0,0.43,0,0.1,0.25,0.19,0.1,0])



#Bounds of raws and macronutrients
supbd=10

bound=np.array([0.4, 0.4, 0.4, 0.05, 0.22, 1,0.04, 0.08,1,0.0065, 0.06, 0.04, 0.05, 0.1, 0.15, 0.2, 1])
Q=np.concatenate([CF, Ca, AP, DM, CP, MC, T, Tp, Energy, L]).reshape(10,17)
lower=np.array([0, 0.6, 0.15, 87, 18, 0.475, 0.627, 0.171, 14.001, 0.9501])
upper=np.array([6, supbd, supbd, 95, 21, supbd,supbd, supbd, 20,2])

#number of variables and standard deviation, and variances
n=17

ones=np.ones(n)

stdl = np.array([0.03962323, 0.0484768 , 0.01870829, 0.08031189, 0.02408319,0.13823892, 0.54625086, 0.09300538, 0., 0.,0.06356099, 0. , 0.08848729, 0.08757854, 0.06511528,0.03 , 0. ])
stde=np.array([0.3197, 0.4066, 0.2644, 1.0968, 0.5168, 0.3607, 1.4353, 0.5676,0.    , 0.    , 0.7056, 0.    , 1.4691, 0.5844, 0.3746, 0.2462,0.    ])

vare=np.diag(stde**2)
varl=np.diag(stdl**2)

In [17]:
def gensamples(nn):
    #rgn=np.random.seed(1234)
    lista=[]
    for i in range(nn):
        w=cp.Variable(n, pos=True)
        cons=cp.Constant(np.random.normal(160,2))
    

        objective_fct = cp.Maximize(1)


        constraints=[ones@w==1, Q@w>=lower, Q@w<=upper,
                     w[0]<=bound[0],
                     w[1]<=bound[1], 
                     w[2]<=bound[2],
                     w[3]<=bound[3],
                     w[4]<=bound[4], 
                     w[5]<=bound[5],
                     w[6]<=bound[6],
                     w[7]<=bound[7],
                     w[8]<=bound[8],
                     w[9]<=bound[9], 
                     w[10]<=bound[10],
                     w[11]<=bound[11],
                     w[12]<=bound[12], 
                     w[13]<=bound[13],
                     w[14]<=bound[14],
                     w[15]<=bound[15],
                     w[16]<=bound[16],
                     Costos@w.T-cons<=0]
        prob = cp.Problem(objective_fct, constraints)
        assert  prob.is_dqcp()
        prob.solve(qcp=True, solver='ECOS')
        #xvalue=w.value/t.value
        lista.append(w.value)
    return np.array(lista)

In [7]:
generate_initial_data(10)

NameError: name 'generate_initial_data' is not defined

In [20]:
Costos=torch.as_tensor(Costos).to(**tkwargs)

CF=torch.as_tensor(CF).to(**tkwargs)
Ca=torch.as_tensor(Ca).to(**tkwargs)
AP=torch.as_tensor(AP).to(**tkwargs)
DM=torch.as_tensor(DM).to(**tkwargs)
CP=torch.as_tensor(CP).to(**tkwargs)
MC=torch.as_tensor(MC).to(**tkwargs)
Tp=torch.as_tensor(Tp).to(**tkwargs)
T=torch.as_tensor(T).to(**tkwargs)
L=torch.as_tensor(L).to(**tkwargs)
Energy=torch.as_tensor(Energy).to(**tkwargs)
ones=torch.as_tensor(ones).to(**tkwargs)

stdl =torch.as_tensor(stdl).to(**tkwargs)
stde=torch.as_tensor(stde).to(**tkwargs)


vare=torch.diag(stde**2)
varl=torch.diag(stdl**2)


In [21]:
class myfunction(MultiObjectiveTestProblem):

    
    dim = 17
    num_objectives=3

    lower=torch.zeros(17)
    upper=torch.as_tensor(bound)
 
    _bounds=torch.stack([lower, upper])
   
    _ref_point=[-170,0,0]
    
    
    def __init__(self, noise_std: Optional[float] = None, negate: bool = False) -> None:
        r"""
        Args:
            noise_std: Standard deviation of the observation noise.
            negate: If True, negate the objectives.
        """
        super().__init__(noise_std=noise_std, negate=negate)

    def _f1(self, X:Tensor)-> Tensor:
          
        return -Costos@torch.transpose(X,1,0)

    def _f2(self, X:Tensor)-> Tensor:
           
        return  L@torch.transpose(X,1,0)
    
    def  _f3(self, X:Tensor)-> Tensor:
       
        return Energy@torch.transpose(X,1,0)
    
    def evaluate_true(self, X: Tensor) -> Tensor:
        F1 = self._f1(X=X)
        F2 = self._f2(X=X)
        F3 = self._f3(X=X)
        return torch.stack([F1,F2,F3],dim=-1)

In [22]:
problem=myfunction().to(**tkwargs)

In [23]:
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from botorch.utils.sampling import draw_sobol_samples

NOISE_SE = torch.tensor([0.1,0.1,0.1], **tkwargs)

def generate_initial_data(n):
    # generate training data
    train_x = torch.tensor(gensamples(n)).to(**tkwargs)
    train_obj_true = problem(train_x)
    train_obj = train_obj_true + torch.randn_like(train_obj_true) * NOISE_SE
    
    return train_x, train_obj

def initialize_model(train_x, train_obj):
    # define models for objective and constraint
    train_x = normalize(train_x, problem._bounds)
    models = []
    for i in range(train_obj.shape[-1]):
        train_y = train_obj[..., i:i+1]
        train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
        models.append(
            FixedNoiseGP(train_x, train_y, train_yvar, outcome_transform=Standardize(m=1))
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model

In [24]:
from botorch import fit_gpytorch_model
from botorch.sampling import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning

import time
import warnings

In [25]:
ineqconstr=[(torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), -CF, -6),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), Ca, 0.6),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), -Ca, -supbd),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), AP, 0.15),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), -AP, -supbd),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), DM, 87),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), -DM, -95),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), CP, 18),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), -CP, -21),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), MC, 0.475),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), Tp, 0.171),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), T, 0.627),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), -MC, -supbd),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), -Tp, -supbd),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), -T, -supbd),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), Energy, 14.001),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), -Energy, -20),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), L, 0.9501),
            (torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), -L, -2),
            
           ]
eqconstr=[(torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]), ones, 1),]

In [26]:
from botorch.acquisition.objective import GenericMCObjective, IdentityMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement, qNoisyExpectedHypervolumeImprovement
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list

In [242]:
def optimize_qehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, problem._bounds)).mean
        partitioning = FastNondominatedPartitioning(
        ref_point=problem.ref_point, 
        Y=pred,
    )
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=problem.ref_point.tolist(),
        partitioning=partitioning,
        #X_baseline=normalize(train_x, problem._bounds),
        #inequality_constraints=ineqconstr,
        #equality_constraints=eqconstr,
        sampler=sampler,
        
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=problem._bounds,
        q=1,
        num_restarts=10,
        raw_samples=256,
        #batch_initial_conditions=train_x.reshape(b,1,17),
        inequality_constraints=ineqconstr,
        equality_constraints=eqconstr,
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
        
    # observe new values 
    new_x =  candidates
    new_obj = problem(new_x)
    #new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj, #new_obj_true

In [14]:
import pickle

In [ ]:


# Guardar los resultados en un archivo utilizando pickle
with open('resultados.pkl', 'wb') as archivo:
    pickle.dump(resultados, archivo)


In [ ]:
Values=[10*i for i in range(1,13)]
EXP1=[]
for value in Values:
    EXP=[]
    print(f'value={value}')
    
    seeds = np.random.randint(0, 1000,30)
    print(seeds)

    for i in range(30):
        N_BATCH = 50 if not SMOKE_TEST else 10
        MC_SAMPLES = 256 if not SMOKE_TEST else 16

        verbose = True

        #seed=92
       
        seed=int(seeds[i])
        print(f"Seed: {seed}")   
        np.random.seed(seed)
        torch.manual_seed(seed)

        hvs_ehvi=[]
        nume=[]

        # call helper functions to generate initial training data and initialize model
        train_x_ehvi, train_obj_ehvi = generate_initial_data(n=value)
        #train_x_ehvi=train_x_ehvi1 
        #train_obj_ehvi=train_obj_ehvi1
        mll_ehvi, model_ehvi = initialize_model(train_x_ehvi, train_obj_ehvi)


        # compute hypervolume
        bd = FastNondominatedPartitioning(ref_point=problem.ref_point, Y=train_obj_ehvi)
        volume = bd.compute_hypervolume().item()
        nume.append(bd.pareto_Y.shape[0])


        hvs_ehvi.append(volume)

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):    

            t0 = time.monotonic()

            # fit the models

            fit_gpytorch_model(mll_ehvi)


            # define the qEI and qNEI acquisition modules using a QMC sampler

            ehvi_sampler = SobolQMCNormalSampler(MC_SAMPLES, seed=seed)


            # optimize acquisition functions and get new observations

            new_x_ehvi, new_obj_ehvi= optimize_qehvi_and_get_observation(
                model_ehvi,train_x_ehvi, train_obj_ehvi, ehvi_sampler
            )


            train_x_ehvi = torch.cat([train_x_ehvi, new_x_ehvi])
            train_obj_ehvi = torch.cat([train_obj_ehvi, new_obj_ehvi])





            #compute hypervolume
            bd = FastNondominatedPartitioning(ref_point=problem.ref_point, Y=train_obj_ehvi)
            volume = bd.compute_hypervolume().item()
            hvs_ehvi.append(volume)
            nume.append(bd.pareto_Y.shape[0])




            mll_ehvi, model_ehvi = initialize_model(train_x_ehvi, train_obj_ehvi)


            t1 = time.monotonic()
            EXP.append([value,seed, volume, bd.pareto_Y.shape[0],bd.pareto_Y, train_x_ehvi, train_obj_ehvi])
        
            if verbose:
                print(
                     f"\nBatch {iteration:>2}:", 
                     f"Hypervolume (qEHVI) = {hvs_ehvi[-1]:>4.4f}", 
                     f"#Paprox={bd.pareto_Y.shape[0]}",   
                     f"time = {t1-t0:>4.2f}", 
                     f"Lysine= {(new_x_ehvi[-1]@L.T).item():>4.4f}",
                     f"Energy= {(new_x_ehvi[-1]@Energy).item():>4.4f}",
                     f"Costs= {(new_x_ehvi[-1]@Costos.T).item():>4.4f}", end=")")
            else:
                print(".", end="")
            
    
        lista = globals()[f'EXP']
        filename = f'EXP_{value}.pickle'
        with open(filename, 'wb') as file:
            pickle.dump(lista, file)

 

value=10
[495 468  97 793 665 271 844 907 189  45 257 205 742 364 855 272 192 622
 967 462 253 497 679 166 805 357 587 381 121 933]
Seed: 495

Batch  1: Hypervolume (qEHVI) = 240.2393 #Paprox=6 time = 1.82 Lysine= 1.3704 Energy= 14.5022 Costs= 160.2403)
Batch  2: Hypervolume (qEHVI) = 240.5544 #Paprox=7 time = 1.65 Lysine= 1.3677 Energy= 14.4851 Costs= 160.0335)
Batch  3: Hypervolume (qEHVI) = 242.7725 #Paprox=8 time = 1.70 Lysine= 1.3952 Energy= 14.7088 Costs= 162.5482)
Batch  4: Hypervolume (qEHVI) = 242.7893 #Paprox=9 time = 1.69 Lysine= 1.3950 Energy= 14.7104 Costs= 162.5573)
Batch  5: Hypervolume (qEHVI) = 242.8203 #Paprox=10 time = 1.95 Lysine= 1.4011 Energy= 14.6969 Costs= 162.5287)
Batch  6: Hypervolume (qEHVI) = 242.8203 #Paprox=10 time = 1.95 Lysine= 1.3426 Energy= 14.3778 Costs= 162.4451)
Batch  7: Hypervolume (qEHVI) = 242.8203 #Paprox=10 time = 2.06 Lysine= 1.3404 Energy= 14.3896 Costs= 162.3852)
Batch  8: Hypervolume (qEHVI) = 242.8203 #Paprox=10 time = 2.66 Lysine= 1.730

In [27]:
def optimize_qNehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, problem._bounds)).mean
        partitioning = FastNondominatedPartitioning(
        ref_point=problem.ref_point, 
        Y=pred,
    )
    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=problem.ref_point.tolist(),
        partitioning=partitioning,
        X_baseline=normalize(train_x, problem._bounds),
        #inequality_constraints=ineqconstr,
        #equality_constraints=eqconstr,
        sampler=sampler,
        
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=problem._bounds,
        q=1,
        num_restarts=10,
        raw_samples=256,
        #batch_initial_conditions=train_x.reshape(b,1,17),
        inequality_constraints=ineqconstr,
        equality_constraints=eqconstr,
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
        
    # observe new values 
    new_x =  candidates
    new_obj = problem(new_x)
    #new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj, #new_obj_true

In [ ]:
Values=[10*i for i in range(1,13)]
EXP1=[]
for value in Values:
    EXP=[]
    print(f'value={value}')
    
    seeds = np.random.randint(0, 1000,30)
    print(seeds)

    for i in range(30):
        N_BATCH = 50 if not SMOKE_TEST else 10
        MC_SAMPLES = 256 if not SMOKE_TEST else 16

        verbose = True

        #seed=92
       
        seed=int(seeds[i])
        print(f"Seed: {seed}")   
        np.random.seed(seed)
        torch.manual_seed(seed)

        hvs_ehvi=[]
        nume=[]

        # call helper functions to generate initial training data and initialize model
        train_x_ehvi, train_obj_ehvi = generate_initial_data(n=value)
        #train_x_ehvi=train_x_ehvi1 
        #train_obj_ehvi=train_obj_ehvi1
        mll_ehvi, model_ehvi = initialize_model(train_x_ehvi, train_obj_ehvi)


        # compute hypervolume
        bd = FastNondominatedPartitioning(ref_point=problem.ref_point, Y=train_obj_ehvi)
        volume = bd.compute_hypervolume().item()
        nume.append(bd.pareto_Y.shape[0])


        hvs_ehvi.append(volume)

        # run N_BATCH rounds of BayesOpt after the initial random batch
        for iteration in range(1, N_BATCH + 1):    

            t0 = time.monotonic()

            # fit the models

            fit_gpytorch_model(mll_ehvi)


            # define the qEI and qNEI acquisition modules using a QMC sampler

            ehvi_sampler = SobolQMCNormalSampler(MC_SAMPLES, seed=seed)


            # optimize acquisition functions and get new observations

            new_x_ehvi, new_obj_ehvi= optimize_qNehvi_and_get_observation(
                model_ehvi,train_x_ehvi, train_obj_ehvi, ehvi_sampler
            )


            train_x_ehvi = torch.cat([train_x_ehvi, new_x_ehvi])
            train_obj_ehvi = torch.cat([train_obj_ehvi, new_obj_ehvi])





            #compute hypervolume
            bd = FastNondominatedPartitioning(ref_point=problem.ref_point, Y=train_obj_ehvi)
            volume = bd.compute_hypervolume().item()
            hvs_ehvi.append(volume)
            nume.append(bd.pareto_Y.shape[0])




            mll_ehvi, model_ehvi = initialize_model(train_x_ehvi, train_obj_ehvi)


            t1 = time.monotonic()
            EXP.append([value,seed, volume, bd.pareto_Y.shape[0],bd.pareto_Y, train_x_ehvi, train_obj_ehvi])
        
            if verbose:
                print(
                     f"\nBatch {iteration:>2}:", 
                     f"Hypervolume (qEHVI) = {hvs_ehvi[-1]:>4.4f}", 
                     f"#Paprox={bd.pareto_Y.shape[0]}",   
                     f"time = {t1-t0:>4.2f}", 
                     f"Lysine= {(new_x_ehvi[-1]@L.T).item():>4.4f}",
                     f"Energy= {(new_x_ehvi[-1]@Energy).item():>4.4f}",
                     f"Costs= {(new_x_ehvi[-1]@Costos.T).item():>4.4f}", end=")")
            else:
                print(".", end="")
            
    
        lista = globals()[f'EXP']
        filename = f'EXP_{value}N.pickle'
        with open(filename, 'wb') as file:
            pickle.dump(lista, file)

 

value=10
[247  24 314 726 196 444 290 443 427 759 958  39 840 120 114 190 744 741
 958 111 441 356 779 250 584 321 138 928 519 622]
Seed: 247


/tmp/ipykernel_2234934/4139669193.py:91: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3277.)
  f"Lysine= {(new_x_ehvi[-1]@L.T).item():>4.4f}",



Batch  1: Hypervolume (qEHVI) = 407.1694 #Paprox=7 time = 5.21 Lysine= 0.9501 Energy= 14.3911 Costs= 146.0545)
Batch  2: Hypervolume (qEHVI) = 407.1694 #Paprox=7 time = 5.39 Lysine= 1.2645 Energy= 14.0772 Costs= 164.8840)
Batch  3: Hypervolume (qEHVI) = 408.9592 #Paprox=8 time = 6.02 Lysine= 0.9501 Energy= 14.4715 Costs= 146.5670)
Batch  4: Hypervolume (qEHVI) = 408.9592 #Paprox=9 time = 6.87 Lysine= 0.9501 Energy= 14.4715 Costs= 146.5670)

/home/in2lab/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(



Batch  5: Hypervolume (qEHVI) = 433.1407 #Paprox=10 time = 7.45 Lysine= 0.9501 Energy= 14.1151 Costs= 144.2513)

...

/home/in2lab/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(



Batch 32: Hypervolume (qEHVI) = 489.5172 #Paprox=39 time = 31.44 Lysine= 1.1197 Energy= 14.0010 Costs= 148.7234)

/home/in2lab/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(



Batch 33: Hypervolume (qEHVI) = 489.5172 #Paprox=40 time = 31.08 Lysine= 1.1197 Energy= 14.0010 Costs= 148.7234)

/home/in2lab/.local/lib/python3.10/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
